# Creating and Valuing a CDS Index Option

This notebook does the valuation of an option on a CDS index using Anderson's method and an Adjusted Black Method. For details see Modelling Singlename and Multiname Credit Derivatives by D.O'Kane.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.200 - This build:  14 Jul 2021 at 13:18 #
# **** NEW PEP8 COMPLIANT VERSION -- PLEASE UPDATE YOUR CODE  **** #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



In [3]:
valuation_date = Date(1, 8, 2007)
step_in_date = valuation_date.add_weekdays(0)
valuation_date = step_in_date
settlement_date = step_in_date

## Build Libor Curve

In [5]:
dcType = DayCountTypes.THIRTY_E_360_ISDA
depos = []
depo = IborDeposit(settlement_date, "1D", 0.0500, dcType); depos.append(depo)

fixedFreq = FrequencyTypes.SEMI_ANNUAL
swapType = SwapTypes.PAY
swap1 = IborSwap(settlement_date,"1Y",swapType,0.0502,fixedFreq,dcType)
swap2 = IborSwap(settlement_date,"2Y",swapType,0.0502,fixedFreq,dcType)
swap3 = IborSwap(settlement_date,"3Y",swapType,0.0501,fixedFreq,dcType)
swap4 = IborSwap(settlement_date,"4Y",swapType,0.0502,fixedFreq,dcType)
swap5 = IborSwap(settlement_date,"5Y",swapType,0.0501,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5]

libor_curve = IborSingleCurve(valuation_date, depos, [], swaps)

In [6]:
cdsMaturity3Y = valuation_date.next_cds_date(36)
cdsMaturity5Y = valuation_date.next_cds_date(60)
cdsMaturity7Y = valuation_date.next_cds_date(84)
cdsMaturity10Y = valuation_date.next_cds_date(120)

## Load the Underlying CDS Index Portfolio

In [9]:
f = open('.//data//CDX_NA_IG_S7_SPREADS.csv', 'r')
data = f.readlines()
issuer_curves = []

num_credits = len(data) - 1  # The file has a header

for row in data[1:]:
    splitRow = row.split(",")
    creditName = splitRow[0]
    spd3Y = float(splitRow[1]) / 10000.0
    spd5Y = float(splitRow[2]) / 10000.0
    spd7Y = float(splitRow[3]) / 10000.0
    spd10Y = float(splitRow[4]) / 10000.0
    recovery_rate = float(splitRow[5])

    cds3Y = CDS(step_in_date, cdsMaturity3Y, spd3Y)
    cds5Y = CDS(step_in_date, cdsMaturity5Y, spd5Y)
    cds7Y = CDS(step_in_date, cdsMaturity7Y, spd7Y)
    cds10Y = CDS(step_in_date, cdsMaturity10Y, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]

    issuer_curve = CDSCurve(valuation_date, cds_contracts, libor_curve, recovery_rate)
    issuer_curves.append(issuer_curve)

## Term Structure of CDS Index Market

We now set up the term structure of CDS Index trades and start by setting up the maturity dates of the index

In [10]:
indexMaturity3Y = Date(20,12,2009)
indexMaturity5Y = Date(20,12,2011)
indexMaturity7Y = Date(20,12,2013)
indexMaturity10Y = Date(20,12,2016)

Now we set their upfronts and coupons

In [11]:
indexUpfronts = [0.0, 0.0, 0.0, 0.0]
indexMaturityDates = [indexMaturity3Y, indexMaturity5Y, indexMaturity7Y, indexMaturity10Y]
indexRecovery = 0.40

## Setting up the Index Option

In [12]:
expiry_date = Date(1,2,2008)
maturity_date = Date(20,12,2011)
notional = 10000
volatility = 0.50
index_coupon = 0.004
tolerance = 1e-6

## Comparison of Anderson versus Adjusted Black Model

In [15]:
print("STRIKE    INDEX      PAY    RECEIVER      G(K)       X         EXPH       ABPAY       ABREC")

for index in np.linspace(20, 60, 5):
    
    # Create a flat CDS index curve
    cds_contracts = []
    for dt in indexMaturityDates:
        cds = CDS(valuation_date, dt, index / 10000.0)
        cds_contracts.append(cds)

    # Build the flat CDS index curve
    index_curve = CDSCurve(valuation_date, cds_contracts, libor_curve, indexRecovery)

    # Now we need to set up the underlying curves and adjust them
    indexSpreads = [index / 10000.0] * 4

    indexPortfolio = CDSIndexPortfolio()
    adjustedIssuerCurves = indexPortfolio.hazard_rate_adjust_intrinsic(valuation_date, 
                                                                    issuer_curves, 
                                                                    indexSpreads, 
                                                                    indexUpfronts,
                                                                    indexMaturityDates, 
                                                                    indexRecovery, 
                                                                    tolerance)        
        
    for strike in np.linspace(20, 60, 5):    

        option = CDSIndexOption(expiry_date,
                                   maturity_date, 
                                   index_coupon, 
                                   strike / 10000.0, 
                                   notional)
        
        v_pay_1, v_rec_1, strikeValue, mu, expH = option.value_anderson(valuation_date, 
                                                                       adjustedIssuerCurves, 
                                                                       indexRecovery,
                                                                       volatility)

        v_pay_2, v_rec_2 = option.value_adjusted_black(valuation_date,
                                                     index_curve, 
                                                     indexRecovery, 
                                                     libor_curve,
                                                     volatility)
            
        print("%6.3f   %6.3f   %8.3f   %8.3f   %8.3f   %8.2f   %8.2f   %8.2f   %8.2f" % 
              (strike, index, v_pay_1, v_rec_1, strikeValue, mu, expH, v_pay_2, v_rec_2 ))

STRIKE    INDEX      PAY    RECEIVER      G(K)       X         EXPH       ABPAY       ABREC
20.000   20.000     16.167      6.203    -70.749      22.90     -60.53      16.19       6.11
30.000   20.000      4.074     28.782    -35.263      22.89     -60.59       4.07      28.63
40.000   20.000      0.896     60.057      0.000      22.87     -60.65       0.91      59.84
50.000   20.000      0.191     93.588     35.040      22.85     -60.71       0.20      93.22
60.000   20.000      0.044    127.460     69.858      22.84     -60.77       0.05     126.88
20.000   30.000     50.470      0.887    -70.749      34.40     -19.91      50.54       0.84
30.000   30.000     24.146      9.206    -35.263      34.39     -19.94      24.18       9.15
40.000   30.000      9.808     29.291      0.000      34.38     -19.97       9.89      29.26
50.000   30.000      3.683     57.372     35.040      34.37     -20.00       3.75      57.23
60.000   30.000      1.340     89.020     69.858      34.37     -20.03 

In [16]:
print(option)

OBJECT TYPE: CDSIndexOption
EXPIRY DATE: 01-FEB-2008
MATURITY DATE: 20-DEC-2011
INDEX COUPON: 40.0bp
NOTIONAL: 10000
LONG PROTECTION: True
FREQUENCY: FrequencyTypes.QUARTERLY
DAYCOUNT: DayCountTypes.ACT_360
CALENDAR: CalendarTypes.WEEKEND
BUSDAYRULE: BusDayAdjustTypes.FOLLOWING
DATEGENRULE: DateGenRuleTypes.BACKWARD



Copyright (c) 2020 Dominic O'Kane